# Atlas Vector Search - Haystack Integration

This notebook is a companion to the [Haystack Get Started](https://www.mongodb.com/docs/atlas/atlas-vector-search/ai-integrations/haystack/) page. Refer to the page for set-up instructions and detailed explanations.

<a target="_blank" href="https://colab.research.google.com/github/mongodb/docs-notebooks/blob/main/ai-integrations/haystack.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
pip install --quiet --upgrade mongodb-atlas-haystack pymongo

In [ ]:
import os
from haystack import Pipeline, Document
from haystack.document_stores.types import DuplicatePolicy
from haystack.components.writers import DocumentWriter
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.embedders import OpenAITextEmbedder, OpenAIDocumentEmbedder
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever
from pymongo import MongoClient
from pymongo.operations import SearchIndexModel

In [ ]:
os.environ["OPENAI_API_KEY"] = "<api-key>"
os.environ["MONGO_CONNECTION_STRING"]= "<connection-string>"

In [ ]:
client = MongoClient(os.environ.get("MONGO_CONNECTION_STRING"))

In [ ]:
# Create your database and collection
db_name = "haystack_db"
collection_name = "test"
database = client[db_name]
database.create_collection(collection_name)

# Define collection
collection = client[db_name][collection_name]

In [ ]:
# Create your index model, then create the search index
search_index_model = SearchIndexModel(
   definition={
      "fields": [
         {
            "type": "vector",
            "path": "embedding",
            "numDimensions": 1536,
            "similarity": "cosine"
         }
      ]
   },
   name="vector_index",
   type="vectorSearch"
)

collection.create_search_index(model=search_index_model)

In [ ]:
document_store = MongoDBAtlasDocumentStore(
   database_name="haystack_db",
   collection_name="test",
   vector_search_index="vector_index",
)

In [ ]:
# Create some example documents
documents = [
   Document(content="My name is Jean and I live in Paris."),
   Document(content="My name is Mark and I live in Berlin."),
   Document(content="My name is Giorgio and I live in Rome."),
]

# Initializing a document embedder to convert text content into vectorized form.
doc_embedder = OpenAIDocumentEmbedder()

# Setting up a document writer to handle the insertion of documents into the MongoDB collection.
doc_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.SKIP)

# Creating a pipeline for indexing documents. The pipeline includes embedding and writing documents.
indexing_pipe = Pipeline()
indexing_pipe.add_component(instance=doc_embedder, name="doc_embedder")
indexing_pipe.add_component(instance=doc_writer, name="doc_writer")

# Connecting the components of the pipeline for document flow.
indexing_pipe.connect("doc_embedder.documents", "doc_writer.documents")

# Running the pipeline with the list of documents to index them in MongoDB.
indexing_pipe.run({"doc_embedder": {"documents": documents}})

## Basic RAG

In [ ]:
# Template for generating prompts.
prompt_template = """
    You are an assistant allowed to use the following context documents.\nDocuments:
    {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}

    \nQuery: {{query}}
    \nAnswer:
"""

# Setting up a retrieval-augmented generation (RAG) pipeline for generating responses.
rag_pipeline = Pipeline()
rag_pipeline.add_component("text_embedder", OpenAITextEmbedder())

# Adding a component for retrieving related documents from MongoDB based on the query embedding.
rag_pipeline.add_component(instance=MongoDBAtlasEmbeddingRetriever(document_store=document_store,top_k=15), name="retriever")

# Building prompts based on retrieved documents to be used for generating responses.
rag_pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")

# Adding a language model generator to produce the final text output.
rag_pipeline.add_component(instance=OpenAIGenerator(), name="llm")

# Connecting the components of the RAG pipeline to ensure proper data flow.
rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

# Run the pipeline
query = "Where does Mark live?"
result = rag_pipeline.run(
  {
      "text_embedder": {"text": query},
      "prompt_builder": {"query": query},
  });
print(result['llm']['replies'][0])